# Lab 6: Recurrent neural networks

In this lab you will use a recurrent neural network to predict whether or not a *tweet* is talking about a real disaster or not. To do this, we will use *Kaggle.com*'s competition [Natural Language Processing with Disaster Tweets](https://www.kaggle.com/c/nlp-getting-started). Please follow the competition directions to obtain the data and evaluate your final model, noting the extra requirements below. **There is no requirement to actually submit your results to the competition.**

**Disclaimer: The dataset for this competition contains text that may be considered profane, vulgar, or offensive.**

**Requirements**
 - Keras's `TextVectorization` functionality must be used, although it need not be part of the model
 - `train.csv` should be split into training and validation sets 
 - the heart of your model must only use recurrent layers chosen from those [available in Keras](https://keras.io/api/layers/#recurrent-layers)
 - an embedding layer should be used; this can be learned along with the main task or use the [GloVe](https://github.com/stanfordnlp/GloVe) or [word2vec]() pretrained word embeddings
 - the evaluation metric for this dataset is the [F1-Score](https://www.kaggle.com/c/nlp-getting-started/overview/evaluation)

**Grading:** 

 - 50% of the grade will come from FINAL, error-free code written in Python/Keras that accomplishes all the steps outlined  
 - 50% will come from descriptive comments associated with that code, where the comments explain what the code is doing and why it is important to the overall objective


**What to submit:**
- a copy of this notebook with:
    - final, well-commented, error-free code in Python/Keras
    - all code cells executed and output visible
- a `submission.csv` file containing the predictions of your final model on the `test.csv` data
- the final version of your model saved as a `Group_#_Lab_6.keras` file

**What NOT to submit:**
 - data files



### Group#: 08  

|Name | Student ID | 
|:-:|:-:|
|Rohit Rohit  |  0773987|
|Rakesh Singh  | 0775942 |
|Praveen Thudianplackal Jose  | 0779521 |
|Reshma Thomas  | 0779853 |

In [2]:
# Mounting the google drive to set the path for data accessing
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
## changing the current working directory to the directory where there are our dataset files.
%cd /content/drive/MyDrive 

/content/drive/MyDrive


In [8]:
import pandas as pd     #Import Pandas
traindata = pd.read_csv("train.csv")   # access the dataset file
testdata=pd.read_csv("test.csv")    # access the dataset file

In [9]:
import numpy as np # Importing Numpy Library
 # We have taken the text, target and text column from the dataset and assigned these columns to train_texts, train,_targets, and test_texts in the form of array
 # with the help of numpy array.
train_texts = np.array(traindata['text'])
train_targets = np.array(traindata['target'])
test_texts = np.array(testdata['text'])

In [10]:
# finding the size of train_texts, train_targets, and test_texts  

print(f"Size of training dataset is : {len(train_texts)}")    
print(f"Size of Target dataset is: {len(train_targets)}")
print(f"total texts in test dataset are : {len(test_texts)}")

Size of training dataset is : 7613
Size of Target dataset is: 7613
total texts in test dataset are : 3263


In [11]:
# We have used the train_test_split function from the sklearn library for dividing our dataset. 
# 10 % of the training dataset has been used as a validation data.
# Random state has been defined because of that we will get the same result every time whenever we will run the code chunk.
from sklearn.model_selection import train_test_split
X_train,X_val, y_train,y_val = train_test_split(train_texts,train_targets, test_size = 0.1, random_state = 81)


In [12]:
# importing the required keras libraries to build the model
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras import layers

max_tokens = 10000  # we have pre defined the max_token size, which is 10000
max_length = 600    # max_length also has been defined. i.e 600


#initializing a text_vectorization using our max tokens and max length and we set output mode to int which 
# will convert our words to integers of numbers.
text_vectorization = layers.TextVectorization(
    max_tokens = max_tokens, output_mode = 'int',
    output_sequence_length = max_length
)
text_vectorization.adapt(X_train)
# The adapt method is used for learning the words and assigning them to numbers. Here, we have defined maximum words are 10,000.

In [13]:
#converting train data, validation data and test data into list of numbers using the vocabulory created early using textvectorization

X_train = text_vectorization(X_train)  
X_val = text_vectorization(X_val)
X_test = text_vectorization(test_texts)

In [14]:
embedding_dim = 50  # we have defined there would be 50 numbers for each word.

vocabulary = text_vectorization.get_vocabulary()   # get_vocabulary is used to retrive the computed vocabulary
word_index = dict(zip(vocabulary, range(len(vocabulary))))

In [15]:
import numpy as np

filename = "glove.6B.50d.txt"
embeddings_index = {}

with open(filename, encoding='utf-8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400001 word vectors.


In [16]:
embedding_dim = 50

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

In [17]:
for i, (word, integer) in enumerate(word_index.items()):
    if i == 10: break
    print((word, integer))

('', 0)
('[UNK]', 1)
('the', 2)
('a', 3)
('in', 4)
('to', 5)
('of', 6)
('and', 7)
('i', 8)
('is', 9)


In [18]:
embedding_matrix = np.zeros((max_tokens, embedding_dim))

for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [21]:
# Creating a model using Keras 
input = keras.Input(shape = (None,),dtype = "int64") # the shape is set to None while dtype is define as 'ínt64'
embedded = layers.Embedding(max_tokens, 
                            embedding_dim,
                            embeddings_initializer = keras.initializers.Constant(embedding_matrix),
                            trainable = False,
                            mask_zero = True)(input)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
# adding a bidirectional layer which is a LSTM layer which gets 32 dimensions as output which is fed to the nect layer.

x = layers.Dropout(0.5)(x)
#Here the dropout layer is helping to restrict the overfitting. It is usually using after the layer of the model

output = layers.Dense(1, activation = "sigmoid")(x)
#The dense is used for creating the fully connected layers, and sigmoid Activation is being used 

model = keras.Model(inputs = input, outputs = output)

model.compile(optimizer = 'rmsprop',
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy'])
#While doing the model compiling, we have used optimizer as 'rmsprop', loss as 'binary_crossentropy' and 
#the metrics for evaluation is choosed as 'accuracy'.

model.summary()   #Summary method is used to show all the details of the model. for example The layers, output shape and Parameter

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 50)          500000    
                                                                 
 bidirectional_4 (Bidirectio  (None, 64)               21248     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 521,313
Trainable params: 21,313
Non-trainable params: 500,000
__________________________________________________

In [23]:
callbacks = keras.callbacks.ModelCheckpoint(filepath = "/Group_8_Lab_6.keras",save_best_only = True)  
# A file has been created to store the model with the help of keras ModelCheckpoint method.

model.fit(X_train,y_train, validation_data=(X_val,y_val), epochs = 10, callbacks = callbacks)

Epoch 1/10
215/215 [==============================] - 107s 500ms/step - loss: 0.3699 - accuracy: 0.8451 - val_loss: 0.4698 - val_accuracy: 0.8084
Epoch 2/10
215/215 [==============================] - 107s 500ms/step - loss: 0.3577 - accuracy: 0.8499 - val_loss: 0.4549 - val_accuracy: 0.7966
Epoch 3/10
215/215 [==============================] - 107s 499ms/step - loss: 0.3495 - accuracy: 0.8537 - val_loss: 0.4520 - val_accuracy: 0.8018
Epoch 4/10
215/215 [==============================] - 107s 498ms/step - loss: 0.3446 - accuracy: 0.8593 - val_loss: 0.4467 - val_accuracy: 0.8123
Epoch 5/10
215/215 [==============================] - 107s 498ms/step - loss: 0.3372 - accuracy: 0.8602 - val_loss: 0.4679 - val_accuracy: 0.7979
Epoch 6/10
215/215 [==============================] - 107s 498ms/step - loss: 0.3255 - accuracy: 0.8688 - val_loss: 0.4779 - val_accuracy: 0.8031
Epoch 7/10
215/215 [==============================] - 107s 497ms/step - loss: 0.3148 - accuracy: 0.8692 - val_loss: 0.4733 -

In [25]:
model1 = keras.models.load_model('/Group_8_Lab_6.keras')  # Loading the file 'Group_8_Lab_6.keras'
predictions = model1.predict(X_test)  # with the help of predict method we are going to predict for the dataset
preds = [1 if i>0.5 else 0 for i in predictions]


In [26]:
submissions = pd.DataFrame()  # A dataframe has been created by using pandas
submissions['ID']= testdata['id'].to_list() # In the dataframe we have defined a column 'ID' 
submissions['Target']=preds  #and another column is 'target' which has been given preds value.
submissions.to_csv('submission.csv', index = False) # finally the dataframe,submission, has been stored as csv file 
submissions.head()

ID  Target
0   0       0
1   2       1
2   3       1
3   9       1
4  11       1